In [1]:
import tensorflow as tf
import numpy as np
import Ipynb_importer
import matplotlib.pyplot as plt
%matplotlib inline

print(tf.__version__)

1.13.1


In [2]:
batch_size = 8*3
folders =   ["F:/Data/data_C1/neg/",  "F:/Data/data_C1/pos/",
            "F:/Data/data_C2/neg/",  "F:/Data/data_C2/pos/"]
# 由于 data_C2的样本中存在约百分之二的标注错误，导致训练结果不理想，所以
# 本设计保存的模型参数是再 dataC1 的数据集上训练出来的。

from data_processing import make_dataset
ds, val,   N_train,N_test = make_dataset(folders[:2]  ,  batch_size=4)

importing Jupyter notebook from data_processing.ipynb
folders 2
F:/Data/data_C1/neg/
F:/Data/data_C1/pos/
总样本数： 1000 

num_neg(500),train_neg(400),test_neg(100)
N_train(800),N_test(200)
load_data Done!!!

1 建立数据集： <DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int32)>
2 复用再乱序： <DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int32)>
Instructions for updating:
Colocations handled automatically by placer.
3 函数再批次： <DatasetV1Adapter shapes: ((?, 288, 576, 1), (?,)), types: (tf.float32, tf.int32)>
4 预取数据：   <DatasetV1Adapter shapes: ((?, 288, 576, 1), (?,)), types: (tf.float32, tf.int32)>
make_dataset done!



In [3]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Conv2D,Activation,MaxPool2D,Flatten,Dense,add,Dropout,GlobalAveragePooling2D

# from VggNet import vggnet_model
# model =  vggnet_model()
def block33(x, N, is_cx ,block_name):
    # x shape= [batch_size, high, wide, channel]
    # Hx = Fx(主路3,3结构)+ Cx(侧路1结构)
    bn = -1
    fx = Conv2D(N,(3,3), padding='same', name=block_name+'_Conv1')(x )    # 不直接设置激活函数(AF)     
    fx = tf.layers.BatchNormalization( bn ,name=block_name+'_BN1')(fx)    # 先批标准化(BN)
    fx = Activation('relu',    name  =  block_name  +  '_Relu1'  )(fx)    # 再激活   
    fx = Conv2D(N,(3,3), padding='same', name=block_name+'_Conv2')(fx)    # 主路最后一层不AF，且不BN    
    if is_cx:
        cx = Conv2D(N,  (1,1),  name  =  block_name  +  '_Conv0' )(x )    #  侧路不AF，且不BN
    else:
        cx = x   
    hx = add([fx,cx],name=block_name+'_add')                              # 先把主路和侧路相加
    hx = tf.layers.BatchNormalization( bn,name=block_name+'_BN2' )(hx)    # 再批标准化
    hx = Activation('relu',name=block_name+'_Relu2')(hx)                  # 最后激活
    
    return hx


inputs = Input(  shape  =  [288,576,1],   name='inputs'  )

num = 8
x = Conv2D(num,(7,7),strides=2,padding='same',name='C7x7')(inputs)  
x = tf.layers.BatchNormalization(         name='C7x7_BN' )(x)
x = Activation(    'relu',              name='C7x7_relu' )(x)
x = MaxPool2D(pool_size=(3,3), strides=2,  name='C7x7_MP')(x) 

num = num * 2
block1 = block33(x,      N=num, is_cx=1, block_name='11' )
block1 = block33(block1, N=num, is_cx=0, block_name='12' )
block1 = MaxPool2D(pool_size=(2,2), strides=2, name='1M' )(block1)

num = num * 2
block2 = block33(block1, N=num, is_cx=1, block_name='21' )
block2 = block33(block2, N=num, is_cx=0, block_name='22' )
block2 = MaxPool2D(pool_size=(2,2), strides=2, name='2M' )(block2)

num = num * 2
block3 = block33(block2, N=num, is_cx=1, block_name='31' )
block3 = block33(block3, N=num, is_cx=0, block_name='32' )
block3 = MaxPool2D(pool_size=(2,2), strides=2, name='3M' )(block3)
block3 = Dropout( rate  =  0.5 ,   name  =   '3Droupout' )(block3)

num = num * 2
block4 = block33(block3, N=num, is_cx=1, block_name='41' )
block4 = block33(block4, N=num, is_cx=0, block_name='42' )
block4 = MaxPool2D(pool_size=(2,2), strides=2, name='4M' )(block4)
block4 = Dropout( rate  =  0.5 ,   name  =   '4Droupout' )(block4)

block_out  =  GlobalAveragePooling2D( name   =    'GAP'  )(block4)
# block_out = Flatten()(block4)
block_out  =  Dropout( rate  =  0.5,  name = 'EDroupout' )(block_out)
outputs = Dense(2, activation='softmax',name = 'softmax' )(block_out)

model = tf.keras.Model(inputs, outputs, name='ResNet_18' )

model.summary()
# from tensorflow.keras.utils import plot_model
# plot_model(model,show_shapes=True,rankdir='TB', to_file='ResNet_18.png')

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             (None, 288, 576, 1)  0                                            
__________________________________________________________________________________________________
C7x7 (Conv2D)                   (None, 144, 288, 8)  400         inputs[0][0]                     
__________________________________________________________________________________________________
C7x7_BN (BatchNormalization)    (None, 144, 288, 8)  32          C7x7[0][0]                       
__________________________________________________________________________________________________
C7x7_relu (Activation)          (None, 144, 288, 8)  0           C7x7_BN[0][0]               

In [4]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

print('done!')

done!


In [5]:
model.evaluate( ds   , steps = 800//batch_size*1)                           # 训练数据

33/33 [==============================] - 4s 112ms/step - loss: 0.6952 - acc: 0.5303


[0.695190202106129, 0.530303]

In [6]:
model.load_weights('./save/ResNet/my_model_weights')                        # 加载参数
model.evaluate( ds   , steps = 800//batch_size*1)                           # 训练数据
model.evaluate( val  , steps = 200//batch_size*1)                           # 测试数据

8/8 [==============================] - 1s 83ms/step - loss: 5.3091e-04 - acc: 1.0000


[0.0005309055090911841, 1.0]

In [7]:
# callbacks = [
#   tf.keras.callbacks.TensorBoard(log_dir='.\save\logs\Resnet',histogram_freq = 1,
#                                  batch_size=batch_size,write_grads=True,write_images=True)
# ]
print('done')

model.fit( ds,          epochs=  10  ,        steps_per_epoch = N_train//batch_size)
#            callbacks=callbacks) 
#            validation_data=val_dataset,         validation_steps = 5 )   

# tensorboard --logdir=C:/Users/gang_E540/Desktop/Python/data_C/save/logs/Resnet   # 文件路径不可以用引号

done
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
33/33 [==============================] - 7s 207ms/step - loss: 0.0485 - acc: 0.9848
Epoch 2/10
33/33 [==============================] - 4s 123ms/step - loss: 0.0699 - acc: 0.9848
Epoch 3/10
33/33 [==============================] - 4s 123ms/step - loss: 0.1563 - acc: 0.9773
Epoch 4/10
33/33 [==============================] - 4s 117ms/step - loss: 0.0828 - acc: 0.9773
Epoch 5/10
33/33 [==============================] - 4s 118ms/step - loss: 0.1596 - acc: 0.9242
Epoch 6/10
33/33 [==============================] - 4s 117ms/step - loss: 0.0970 - acc: 0.9848
Epoch 7/10
33/33 [==============================] - 4s 118ms/step - loss: 0.0671 - acc: 0.9848
Epoch 8/10
33/33 [==============================] - 4s 125ms/step - loss: 0.0863 - acc: 0.9773
Epoch 9/10
33/33 [==============================] - 4s 122ms/step - loss: 0.1420 - acc: 0.9697
Epoch 10/10
33/33 [==============================] - 4s 122ms/step - loss: 0.0959 - acc: 0.9

In [8]:
model.evaluate( ds   , steps = N_train//batch_size//3)                           # 训练数据

11/11 [==============================] - 1s 109ms/step - loss: 0.0664 - acc: 0.9773


[0.06635877500155568, 0.97727275]

In [9]:
model.evaluate( val  , steps = N_test //batch_size*3)                           # 测试数据
model.evaluate( ds   , steps = N_train//batch_size*3)                           # 训练数据

99/99 [==============================] - 6s 65ms/step - loss: 0.1031 - acc: 0.9646


[0.10313727380251497, 0.96464646]

In [10]:
predict_test = model.predict(  val ,batch_size=batch_size, steps = 1)
print(predict_test)
predict = np.argmax(predict_test,axis=1)  #axis = 1是取行的最大值的索引，0是列的最大值的索引
# print(predict)
# print(np.sum(predict)/len(predict),len(predict))

[[5.2386237e-04 9.9947613e-01]
 [5.4549711e-04 9.9945456e-01]
 [4.9410253e-03 9.9505901e-01]
 [1.2283486e-20 1.0000000e+00]]


# 训练后不轻易加载和保存

In [11]:
# model.save_weights('./save/ResNet/my_model_weights')
# print('done')

In [12]:
# # 保存模型 + 参数
# model.save('ResNet_18.h5')
# print('done')